In [42]:
from biotrainer.protocols import Protocol

task_protocol = Protocol.sequence_to_class

# Bootstrapping config (sample_size == length of test set)
bootstrapping_confidence_level = 0.05
bootstrapping_iterations = 30

In [43]:
import requests

url = "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_hard.fasta"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
with open("mixed_hard.fasta", "wb") as f:
    f.write(response.content)

In [44]:
from biotrainer.utilities.cli import convert
convert("mixed_hard.fasta") # convert old biotrainer format to new one

In [ ]:
biotrainer_config = {
    "protocol": task_protocol.name,  # We have a per-sequence regression task, so sequence_to_value
    "input_file": "converted.fasta",  # The path to the downloaded fasta file
    "model_choice": "FNN",  # Fully connected neural netowrk
    "embedder_name": "one_hot_encoding",  # Name of the embedding model; alternative: Rostlab/prot_t5_xl_uniref50 
    "num_epochs": 200,  # Define how long we want to train at max
    "loss_choice": "cross_entropy_loss",
    #"bootstrapping_iterations": 30 # adds error bars based on bootstrapping
}


In [46]:
from biotrainer.utilities.cli import train
result=train(biotrainer_config)

''

In [64]:
# First we get the bootstrapping results
bootstrapping_ohe_dict = results["test_results"]["test"]["bootstrapping"]["results"]
ohe_rmse_mean = bootstrapping_ohe_dict["accuracy"]["mean"]
ohe_rmse_ci = bootstrapping_ohe_dict["accuracy"]["error"]


prott5_rmse_mean = 0.2  # Not real data
prott5_rmse_ci = 0.01  # Not real data
# Now let's compare them!
print(f"ProtT5: Accuracy Mean: {prott5_rmse_mean} CI: {prott5_rmse_ci}")
print(f"OHE: Accuracy Mean: {ohe_rmse_mean:.3f} CI: {ohe_rmse_ci:.3f}")

ProtT5: Accuracy Mean: 0.2 CI: 0.01
OHE: Accuracy Mean: 0.310 CI: 0.044
